# Simulation details

* Both are normal distributions

# Expectations

* p value should be less than $\alpha$ when the distributions are different

# Normal distribution

## Same variance

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu, norm, rankdata, wilcoxon, cauchy
import ipywidgets as widgets
from ipywidgets import interact

In [2]:
from tqdm.notebook import tqdm

In [2]:
def plot_interactive_mannwhitneyu(mean1=50, mean2=55, std_dev=10, size=100):
    np.random.seed(42)

    sample1 = np.random.normal(loc=mean1, scale=std_dev, size=size)
    sample2 = np.random.normal(loc=mean2, scale=std_dev, size=size)
    
    stat, p_value = mannwhitneyu(sample1, sample2, alternative='two-sided')
    
    plt.figure(figsize=(10, 6))
    plt.hist(sample1, bins=20, alpha=0.7, label=f'Sample 1 (Mean={mean1})', color='blue', edgecolor='black')
    plt.hist(sample2, bins=20, alpha=0.7, label=f'Sample 2 (Mean={mean2})', color='orange', edgecolor='black')
    plt.axvline(np.mean(sample1), color='blue', linestyle='dashed', linewidth=1)
    plt.axvline(np.mean(sample2), color='orange', linestyle='dashed', linewidth=1)
    plt.title("Histogram of Sample 1 and Sample 2")
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.legend()
    
    plt.figtext(0.15, 0.75, f"Wilcoxon Mann-Whitney Test\nStatistic: {stat:.2f}\nP-value: {p_value:.4f}", 
                fontsize=10, color="black", bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))
    
    plt.show()

interact(plot_interactive_mannwhitneyu, 
         mean1=widgets.FloatSlider(value=50, min=0, max=100, step=1, description='Mean 1'),
         mean2=widgets.FloatSlider(value=55, min=0, max=100, step=1, description='Mean 2'),
         std_dev=widgets.FloatSlider(value=10, min=1, max=30, step=1, description='Std Dev'),
         size=widgets.IntSlider(value=300, min=100, max=500, step=50, description='Sample Size'))


interactive(children=(FloatSlider(value=50.0, description='Mean 1', step=1.0), FloatSlider(value=55.0, descrip…

<function __main__.plot_interactive_mannwhitneyu(mean1=50, mean2=55, std_dev=10, size=100)>

## Different Variance

In [46]:
def plot_interactive_tests(mean1=50, delta=0, std_dev1=10, std_dev2=10, size=100, signif_alpha=0.05, runs=100):
    np.random.seed(42)

    mean2 = mean1 + delta
    mw_p_values, waerden_p_values, median_test_p_values = [], [], []
    fy_p_values, perm_p_values, tstt_p_values = [], [], []
    
    for _ in range(runs):
        sample1 = np.random.normal(loc=mean1, scale=std_dev1, size=size)
        sample2 = np.random.normal(loc=mean2, scale=std_dev2, size=size)
    
        stat_mw, p_value_mw = mannwhitneyu(sample1, sample2, alternative='two-sided')
        combined = np.concatenate([sample1, sample2])
        ranks = rankdata(combined)
        normal_scores = norm.ppf(ranks / (len(combined) + 1))
    
        scores1, scores2 = normal_scores[:size], normal_scores[size:]
        mean_diff = np.abs(np.mean(scores1) - np.mean(scores2))
        pooled_std = np.sqrt(np.var(scores1, ddof=1) / size + np.var(scores2, ddof=1) / size)
        stat_waerden = mean_diff / pooled_std
        p_value_waerden = 2 * (1 - norm.cdf(stat_waerden))
        
        # _, median_p_value = median_test(sample1, sample2)

        # _, fy_p_value = fisher_yates_terry_hoeffding_test(sample1, sample2)

        # perm_p_value, _ = permutation_test(sample1, sample2, alternative='two-sided')

        # _, tstt_p_value = two_sample_t_test(sample1, sample2, equal_var=std_dev1==std_dev2, alternative='two-sided')

        mw_p_values.append(p_value_mw)
        waerden_p_values.append(p_value_waerden)
        # median_test_p_values.append(median_p_value)
        # fy_p_values.append(fy_p_value)
        # perm_p_values.append(perm_p_value)
        # tstt_p_values.append(tstt_p_value)

    fig = plt.figure(figsize=(15, 6))

    # First subplot: Histograms of samples
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.hist(sample1, bins='auto', alpha=0.7, label=f'Sample 1 (Mean={mean1})', color='blue', edgecolor='black')
    ax1.hist(sample2, bins='auto', alpha=0.7, label=f'Sample 2 (Mean={mean2})', color='orange', edgecolor='black')
    ax1.axvline(np.mean(sample1), color='blue', linestyle='dashed', linewidth=1)
    ax1.axvline(np.mean(sample2), color='orange', linestyle='dashed', linewidth=1)
    
    ax1.set_title("Histogram of Sample 1 and Sample 2")
    ax1.set_xlabel("Value")
    ax1.set_ylabel("Frequency")
    ax1.legend()

    # Add text boxes
    ax1.text(0.05, 0.95, f"Mann-Whitney Test\nStatistic: {stat_mw:.2f}\nP-value: {p_value_mw:.4f}\nSignificant: {p_value_mw < signif_alpha}",
             transform=ax1.transAxes, fontsize=10, verticalalignment='top', bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))
    ax1.text(0.05, 0.75, f"Van der Waerden Test\nStatistic: {stat_waerden:.2f}\nP-value: {p_value_waerden:.4f}\nSignificant: {p_value_waerden < signif_alpha}",
             transform=ax1.transAxes, fontsize=10, verticalalignment='top', bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))

    # Second subplot: Violin plots, box plots, and scatter overlay
    ax2 = fig.add_subplot(1, 2, 2)
    data = [mw_p_values, waerden_p_values]
    labels = ['Mann-Whitney', 'Waerden']
    positions = [1, 2]
    
    # Violin plots
    vp = ax2.violinplot(data, positions=positions, showmeans=False, showmedians=False, showextrema=False)
    
    # Customize violin plots
    for i, pc in enumerate(vp['bodies']):
        pc.set_alpha(0.5)
        pc.set_edgecolor('black')
        pc.set_facecolor('lightblue')
    
    # Box plots
    bp = ax2.boxplot(data, positions=positions, widths=0.1, patch_artist=True, 
                     boxprops=dict(facecolor='white', color='black'),
                     medianprops=dict(color='black'),
                     whiskerprops=dict(color='black'),
                     capprops=dict(color='black'))
    
    # Overlay scatter plots
    colors = ['red', 'red']
    for i in range(len(data)):
        y = data[i]
        x = np.random.normal(positions[i], 0.05, size=len(y))
        ax2.plot(x, y, '.', color=colors[i], alpha=0.5)
    
    # Add significance line
    ax2.axhline(y=signif_alpha, color='red', linestyle='--', label=f'Alpha = {signif_alpha}')
    ax2.text(0.05, 0.75, f"MW Proportion: {len([i for i in mw_p_values if i < signif_alpha])/len(mw_p_values)}",
             transform=ax2.transAxes, fontsize=10, verticalalignment='top', bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))
    ax2.text(0.05, 0.95, f"Waerden Proportion: {len([i for i in waerden_p_values if i < signif_alpha])/len(waerden_p_values)}",
             transform=ax2.transAxes, fontsize=10, verticalalignment='top', bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))

    # Set labels and title
    ax2.set_xticks(positions)
    ax2.set_xticklabels(labels)
    ax2.set_title("P-values with Violin and Box Plots")
    ax2.set_ylabel("P-value")
    ax2.legend()

    plt.tight_layout()
    plt.show()

interact(plot_interactive_tests, 
         mean1=widgets.FloatSlider(value=50, min=0, max=100, step=1, description='Mean 1'),
         delta=widgets.FloatSlider(value=0, min=-100, max=100, step=1, description='Delta'),
         std_dev1=widgets.FloatSlider(value=10, min=1, max=30, step=1, description='Std Dev 1'),
         std_dev2=widgets.FloatSlider(value=10, min=1, max=30, step=1, description='Std Dev 2'),
         size=widgets.IntSlider(value=50, min=10, max=200, step=5, description='Sample Size'),
         runs=widgets.IntSlider(value=10, min=10, max=200, step=10, description='Runs'),
         signif_alpha=widgets.FloatSlider(value=0.05, min=0, max=1, step=0.01, description='Alpha'))


interactive(children=(FloatSlider(value=50.0, description='Mean 1', step=1.0), FloatSlider(value=0.0, descript…

<function __main__.plot_interactive_tests(mean1=50, delta=0, std_dev1=10, std_dev2=10, size=100, signif_alpha=0.05, runs=100)>

In [19]:
def plot_interactive_tests(mean1=50, delta=0, std_dev1=10, std_dev2=10, size=100, signif_alpha=0.05, runs=100):
    np.random.seed(42)

    mean2 = mean1 + delta
    mw_p_values, waerden_p_values, median_test_p_values = [], [], []
    fy_p_values, perm_p_values, tstt_p_values = [], [], []
    
    for _ in range(runs):
        sample1 = np.random.normal(loc=mean1, scale=std_dev1, size=size)
        sample2 = np.random.normal(loc=mean2, scale=std_dev2, size=size)
    
        stat_mw, p_value_mw = mannwhitneyu(sample1, sample2, alternative='two-sided')
        combined = np.concatenate([sample1, sample2])
        ranks = rankdata(combined)
        normal_scores = norm.ppf(ranks / (len(combined) + 1))
    
        scores1, scores2 = normal_scores[:size], normal_scores[size:]
        mean_diff = np.abs(np.mean(scores1) - np.mean(scores2))
        pooled_std = np.sqrt(np.var(scores1, ddof=1) / size + np.var(scores2, ddof=1) / size)
        stat_waerden = mean_diff / pooled_std
        p_value_waerden = 2 * (1 - norm.cdf(stat_waerden))
        
        _, median_p_value = median_test(sample1, sample2)
        _, fy_p_value = fisher_yates_terry_hoeffding_test(sample1, sample2)
        perm_p_value, _ = permutation_test(sample1, sample2, alternative='two-sided')
        _, tstt_p_value = two_sample_t_test(sample1, sample2, equal_var=std_dev1 == std_dev2, alternative='two-sided')

        mw_p_values.append(p_value_mw)
        waerden_p_values.append(p_value_waerden)
        median_test_p_values.append(median_p_value)
        fy_p_values.append(fy_p_value)
        perm_p_values.append(perm_p_value)
        tstt_p_values.append(tstt_p_value)

    fig = plt.figure(figsize=(15, 6))

    # First subplot: Histograms of samples
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.hist(sample1, bins='auto', alpha=0.7, label=f'Sample 1 (Mean={mean1:.3f})', color='blue', edgecolor='black')
    ax1.hist(sample2, bins='auto', alpha=0.7, label=f'Sample 2 (Mean={mean2:.3f})', color='orange', edgecolor='black')
    ax1.axvline(np.mean(sample1), color='blue', linestyle='dashed', linewidth=1)
    ax1.axvline(np.mean(sample2), color='orange', linestyle='dashed', linewidth=1)
    ax1.set_title("Histogram of Sample 1 and Sample 2")
    ax1.set_xlabel("Value")
    ax1.set_ylabel("Frequency")
    ax1.legend()

    # Add text boxes
    ax1.text(0.05, 0.95, f"Mann-Whitney Test\nStatistic: {stat_mw:.2f}\nP-value: {p_value_mw:.4f}\nSignificant: {p_value_mw < signif_alpha}",
             transform=ax1.transAxes, fontsize=10, verticalalignment='top', bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))
    ax1.text(0.05, 0.75, f"Van der Waerden Test\nStatistic: {stat_waerden:.2f}\nP-value: {p_value_waerden:.4f}\nSignificant: {p_value_waerden < signif_alpha}",
             transform=ax1.transAxes, fontsize=10, verticalalignment='top', bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))

    # Second subplot: Violin plots, box plots, and scatter overlay
    ax2 = fig.add_subplot(1, 2, 2)
    data = [mw_p_values, waerden_p_values, median_test_p_values, fy_p_values, perm_p_values, tstt_p_values]
    labels = ['Mann-Whitney', 'Waerden', 'Median Test', 'FYTH', 'Permutation Test', 'Two-Sample T-Test']
    positions = list(range(1, len(labels) + 1))
    
    # Violin plots
    vp = ax2.violinplot(data, positions=positions, showmeans=False, showmedians=False, showextrema=False)
    
    # Customize violin plots
    for i, pc in enumerate(vp['bodies']):
        pc.set_alpha(0.5)
        pc.set_edgecolor('black')
        pc.set_facecolor('lightblue')
    
    # Box plots
    bp = ax2.boxplot(data, positions=positions, widths=0.1, patch_artist=True, 
                     boxprops=dict(facecolor='white', color='black'),
                     medianprops=dict(color='black'),
                     whiskerprops=dict(color='black'),
                     capprops=dict(color='black'))
    
    # Overlay scatter plots
    colors = ['red'] * len(data)
    for i, y in enumerate(data):
        x = np.random.normal(positions[i], 0.05, size=len(y))
        ax2.plot(x, y, '.', color=colors[i], alpha=0.5)
    
    # Add significance line
    ax2.axhline(y=signif_alpha, color='red', linestyle='--', label=f'Alpha = {signif_alpha}')

    # Set labels and title
    ax2.set_xticks(positions)
    ax2.set_xticklabels(labels)
    ax2.set_title("P-values with Violin and Box Plots")
    ax2.set_ylabel("P-value")
    ax2.legend()

    plt.tight_layout()
    plt.show()

interact(plot_interactive_tests, 
         mean1=widgets.FloatSlider(value=50, min=0, max=100, step=1, description='Mean 1'),
         delta=widgets.FloatSlider(value=0, min=-50, max=50, step=1, description='Delta'),
         std_dev1=widgets.FloatSlider(value=10, min=1, max=30, step=1, description='Std Dev 1'),
         std_dev2=widgets.FloatSlider(value=10, min=1, max=30, step=1, description='Std Dev 2'),
         size=widgets.IntSlider(value=50, min=10, max=200, step=5, description='Sample Size'),
         runs=widgets.IntSlider(value=10, min=10, max=200, step=10, description='Runs'),
         signif_alpha=widgets.FloatSlider(value=0.05, min=0, max=1, step=0.01, description='Alpha'))


interactive(children=(FloatSlider(value=50.0, description='Mean 1', step=1.0), FloatSlider(value=0.0, descript…

<function __main__.plot_interactive_tests(mean1=50, delta=0, std_dev1=10, std_dev2=10, size=100, signif_alpha=0.05, runs=100)>

In [20]:
def plot_interactive_tests(mean1=50, delta=0, std_dev1=10, std_dev2=10, size=100, signif_alpha=0.05, runs=100,
                           mw_test=True, waerden_test=True, median_t=True, fy_test=True, perm_test=True, t_test=True):
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.stats import mannwhitneyu, rankdata, norm, median_test, ttest_ind
    import ipywidgets as widgets

    np.random.seed(42)
    mean2 = mean1 + delta

    # Initialize p-value lists
    mw_p_values = []
    waerden_p_values = []
    median_test_p_values = []
    fy_p_values = []
    perm_p_values = []
    tstt_p_values = []

    for _ in range(runs):
        sample1 = np.random.normal(loc=mean1, scale=std_dev1, size=size)
        sample2 = np.random.normal(loc=mean2, scale=std_dev2, size=size)

        if mw_test:
            stat_mw, p_value_mw = mannwhitneyu(sample1, sample2, alternative='two-sided')
            mw_p_values.append(p_value_mw)

        if waerden_test:
            combined = np.concatenate([sample1, sample2])
            ranks = rankdata(combined)
            normal_scores = norm.ppf(ranks / (len(combined) + 1))

            scores1, scores2 = normal_scores[:size], normal_scores[size:]
            mean_diff = np.abs(np.mean(scores1) - np.mean(scores2))
            pooled_std = np.sqrt(np.var(scores1, ddof=1) / size + np.var(scores2, ddof=1) / size)
            stat_waerden = mean_diff / pooled_std
            p_value_waerden = 2 * (1 - norm.cdf(stat_waerden))
            waerden_p_values.append(p_value_waerden)

        if median_t:
            _, median_p_value, _, _ = median_test(sample1, sample2)
            median_test_p_values.append(median_p_value)

        if fy_test:
            # Placeholder for the FYTH test; replace with actual implementation
            fy_p_value = np.random.rand()
            fy_p_values.append(fy_p_value)

        if perm_test:
            # Placeholder for the permutation test; replace with actual implementation
            perm_p_value = np.random.rand()
            perm_p_values.append(perm_p_value)

        if t_test:
            _, tstt_p_value = ttest_ind(sample1, sample2, equal_var=std_dev1 == std_dev2)
            tstt_p_values.append(tstt_p_value)

    fig = plt.figure(figsize=(15, 6))

    # First subplot: Histograms of samples
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.hist(sample1, bins='auto', alpha=0.7, label=f'Sample 1 (Mean={mean1:.3f})', color='blue', edgecolor='black')
    ax1.hist(sample2, bins='auto', alpha=0.7, label=f'Sample 2 (Mean={mean2:.3f})', color='orange', edgecolor='black')
    ax1.axvline(np.mean(sample1), color='blue', linestyle='dashed', linewidth=1)
    ax1.axvline(np.mean(sample2), color='orange', linestyle='dashed', linewidth=1)
    ax1.set_title("Histogram of Sample 1 and Sample 2")
    ax1.set_xlabel("Value")
    ax1.set_ylabel("Frequency")
    ax1.legend()

    # Prepare text boxes
    text_boxes = []
    if mw_test:
        text_boxes.append(("Mann-Whitney Test", None, np.mean(mw_p_values), np.mean(mw_p_values) < signif_alpha))
    if waerden_test:
        text_boxes.append(("Van der Waerden Test", None, np.mean(waerden_p_values), np.mean(waerden_p_values) < signif_alpha))
    if median_t:
        text_boxes.append(("Median Test", None, np.mean(median_test_p_values), np.mean(median_test_p_values) < signif_alpha))
    if fy_test:
        text_boxes.append(("FYTH Test", None, np.mean(fy_p_values), np.mean(fy_p_values) < signif_alpha))
    if perm_test:
        text_boxes.append(("Permutation Test", None, np.mean(perm_p_values), np.mean(perm_p_values) < signif_alpha))
    if t_test:
        text_boxes.append(("Two-Sample T-Test", None, np.mean(tstt_p_values), np.mean(tstt_p_values) < signif_alpha))

    # Display text boxes
    if text_boxes:
        vstep = 0.15
        vpos = 0.95
        for test_name, stat, p_value, significant in text_boxes:
            text = f"{test_name}\nMean P-value: {p_value:.4f}\nSignificant: {significant}"
            ax1.text(0.05, vpos, text,
                     transform=ax1.transAxes, fontsize=10, verticalalignment='top',
                     bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))
            vpos -= vstep

    # Second subplot: Violin plots, box plots, and scatter overlay
    ax2 = fig.add_subplot(1, 2, 2)
    data = []
    labels = []
    positions = []
    position_counter = 1

    if mw_test:
        data.append(mw_p_values)
        labels.append('Mann-Whitney')
        positions.append(position_counter)
        position_counter += 1

    if waerden_test:
        data.append(waerden_p_values)
        labels.append('Waerden')
        positions.append(position_counter)
        position_counter += 1

    if median_t:
        data.append(median_test_p_values)
        labels.append('Median Test')
        positions.append(position_counter)
        position_counter += 1

    if fy_test:
        data.append(fy_p_values)
        labels.append('FYTH')
        positions.append(position_counter)
        position_counter += 1

    if perm_test:
        data.append(perm_p_values)
        labels.append('Permutation Test')
        positions.append(position_counter)
        position_counter += 1

    if t_test:
        data.append(tstt_p_values)
        labels.append('Two-Sample T-Test')
        positions.append(position_counter)
        position_counter += 1

    if data:
        # Violin plots
        vp = ax2.violinplot(data, positions=positions, showmeans=False, showmedians=False, showextrema=False)

        # Customize violin plots
        for pc in vp['bodies']:
            pc.set_alpha(0.5)
            pc.set_edgecolor('black')
            pc.set_facecolor('lightblue')

        # Box plots
        bp = ax2.boxplot(data, positions=positions, widths=0.1, patch_artist=True,
                         boxprops=dict(facecolor='white', color='black'),
                         medianprops=dict(color='black'),
                         whiskerprops=dict(color='black'),
                         capprops=dict(color='black'))

        # Overlay scatter plots
        colors = ['red'] * len(data)
        for i, y in enumerate(data):
            x = np.random.normal(positions[i], 0.05, size=len(y))
            ax2.plot(x, y, '.', color=colors[i], alpha=0.5)

        # Add significance line
        ax2.axhline(y=signif_alpha, color='red', linestyle='--', label=f'Alpha = {signif_alpha}')

        # Set labels and title
        ax2.set_xticks(positions)
        ax2.set_xticklabels(labels, rotation=45)
        ax2.set_title("P-values with Violin and Box Plots")
        ax2.set_ylabel("P-value")
        ax2.legend()
    else:
        ax2.text(0.5, 0.5, 'No tests selected.', transform=ax2.transAxes,
                 fontsize=14, ha='center', va='center')
        ax2.axis('off')

    plt.tight_layout()
    plt.show()

import ipywidgets as widgets
from ipywidgets import interact

interact(plot_interactive_tests, 
         mean1=widgets.FloatSlider(value=50, min=0, max=100, step=1, description='Mean 1'),
         delta=widgets.FloatSlider(value=0, min=-50, max=50, step=1, description='Delta'),
         std_dev1=widgets.FloatSlider(value=10, min=1, max=30, step=1, description='Std Dev 1'),
         std_dev2=widgets.FloatSlider(value=10, min=1, max=30, step=1, description='Std Dev 2'),
         size=widgets.IntSlider(value=50, min=10, max=200, step=5, description='Sample Size'),
         runs=widgets.IntSlider(value=10, min=10, max=200, step=10, description='Runs'),
         signif_alpha=widgets.FloatSlider(value=0.05, min=0, max=1, step=0.01, description='Alpha'),
         mw_test=widgets.ToggleButton(value=True, description='Mann-Whitney'),
         waerden_test=widgets.ToggleButton(value=True, description='Waerden'),
         median_t=widgets.ToggleButton(value=True, description='Median Test'),
         fy_test=widgets.ToggleButton(value=True, description='FYTH'),
         perm_test=widgets.ToggleButton(value=True, description='Permutation Test'),
         t_test=widgets.ToggleButton(value=True, description='Two-Sample T-Test'))


interactive(children=(FloatSlider(value=50.0, description='Mean 1', step=1.0), FloatSlider(value=0.0, descript…

<function __main__.plot_interactive_tests(mean1=50, delta=0, std_dev1=10, std_dev2=10, size=100, signif_alpha=0.05, runs=100, mw_test=True, waerden_test=True, median_t=True, fy_test=True, perm_test=True, t_test=True)>

In [25]:
def plot_interactive_tests(mean1=50, delta=0, std_dev1=10, std_dev2=10, size=100, signif_alpha=0.05, runs=100,
                           mw_test=True, waerden_test=True, median_t=True, fy_test=True, perm_test=True, t_test=True):
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.stats import mannwhitneyu, rankdata, norm, median_test, ttest_ind
    import ipywidgets as widgets

    np.random.seed(42)
    mean2 = mean1 + delta

    # Initialize p-value lists
    mw_p_values = []
    waerden_p_values = []
    median_test_p_values = []
    fy_p_values = []
    perm_p_values = []
    tstt_p_values = []

    for _ in range(runs):
        sample1 = np.random.normal(loc=mean1, scale=std_dev1, size=size)
        sample2 = np.random.normal(loc=mean2, scale=std_dev2, size=size)

        if mw_test:
            stat_mw, p_value_mw = mannwhitneyu(sample1, sample2, alternative='two-sided')
            mw_p_values.append(p_value_mw)

        if waerden_test:
            combined = np.concatenate([sample1, sample2])
            ranks = rankdata(combined)
            normal_scores = norm.ppf(ranks / (len(combined) + 1))

            scores1, scores2 = normal_scores[:size], normal_scores[size:]
            mean_diff = np.abs(np.mean(scores1) - np.mean(scores2))
            pooled_std = np.sqrt(np.var(scores1, ddof=1) / size + np.var(scores2, ddof=1) / size)
            stat_waerden = mean_diff / pooled_std
            p_value_waerden = 2 * (1 - norm.cdf(stat_waerden))
            waerden_p_values.append(p_value_waerden)

        if median_t:
            _, median_p_value, _, _ = median_test(sample1, sample2)
            median_test_p_values.append(median_p_value)

        if fy_test:
            # Placeholder for the FYTH test; replace with actual implementation
            fy_p_value = np.random.rand()
            fy_p_values.append(fy_p_value)

        if perm_test:
            # Placeholder for the permutation test; replace with actual implementation
            perm_p_value = np.random.rand()
            perm_p_values.append(perm_p_value)

        if t_test:
            _, tstt_p_value = ttest_ind(sample1, sample2, equal_var=std_dev1 == std_dev2)
            tstt_p_values.append(tstt_p_value)

    fig = plt.figure(figsize=(15, 6))

    # First subplot: Histograms of samples
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.hist(sample1, bins='auto', alpha=0.7, label=f'Sample 1 (Mean={mean1:.3f})', color='blue', edgecolor='black')
    ax1.hist(sample2, bins='auto', alpha=0.7, label=f'Sample 2 (Mean={mean2:.3f})', color='orange', edgecolor='black')
    ax1.axvline(np.mean(sample1), color='blue', linestyle='dashed', linewidth=1)
    ax1.axvline(np.mean(sample2), color='orange', linestyle='dashed', linewidth=1)
    ax1.set_title("Histogram of Sample 1 and Sample 2")
    ax1.set_xlabel("Value")
    ax1.set_ylabel("Frequency")
    ax1.legend()

    # Prepare text boxes
    # text_boxes = []
    # if mw_test:
    #     text_boxes.append(("Mann-Whitney Test", None, np.mean(mw_p_values), np.mean(mw_p_values) < signif_alpha))
    # if waerden_test:
    #     text_boxes.append(("Van der Waerden Test", None, np.mean(waerden_p_values), np.mean(waerden_p_values) < signif_alpha))
    # if median_t:
    #     text_boxes.append(("Median Test", None, np.mean(median_test_p_values), np.mean(median_test_p_values) < signif_alpha))
    # if fy_test:
    #     text_boxes.append(("FYTH Test", None, np.mean(fy_p_values), np.mean(fy_p_values) < signif_alpha))
    # if perm_test:
    #     text_boxes.append(("Permutation Test", None, np.mean(perm_p_values), np.mean(perm_p_values) < signif_alpha))
    # if t_test:
    #     text_boxes.append(("Two-Sample T-Test", None, np.mean(tstt_p_values), np.mean(tstt_p_values) < signif_alpha))

    # # Display text boxes
    # if text_boxes:
    #     vstep = 0.15
    #     vpos = 0.95
    #     for test_name, stat, p_value, significant in text_boxes:
    #         text = f"{test_name}\nMean P-value: {p_value:.4f}\nSignificant: {significant}"
    #         ax1.text(0.05, vpos, text,
    #                  transform=ax1.transAxes, fontsize=10, verticalalignment='top',
    #                  bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))
    #         vpos -= vstep

    # Second subplot: Violin plots, box plots, and scatter overlay
    ax2 = fig.add_subplot(1, 2, 2)
    data = []
    labels = []
    positions = []
    proportions = []
    position_counter = 1

    if mw_test:
        data.append(mw_p_values)
        labels.append('Mann-Whitney')
        positions.append(position_counter)
        prop_mw = len([i for i in mw_p_values if i < signif_alpha]) / len(mw_p_values)
        proportions.append(prop_mw)
        position_counter += 1

    if waerden_test:
        data.append(waerden_p_values)
        labels.append('Waerden')
        positions.append(position_counter)
        prop_waerden = len([i for i in waerden_p_values if i < signif_alpha]) / len(waerden_p_values)
        proportions.append(prop_waerden)
        position_counter += 1

    if median_t:
        data.append(median_test_p_values)
        labels.append('Median Test')
        positions.append(position_counter)
        prop_median = len([i for i in median_test_p_values if i < signif_alpha]) / len(median_test_p_values)
        proportions.append(prop_median)
        position_counter += 1

    if fy_test:
        data.append(fy_p_values)
        labels.append('FYTH')
        positions.append(position_counter)
        prop_fyth = len([i for i in fy_p_values if i < signif_alpha]) / len(fy_p_values)
        proportions.append(prop_fyth)
        position_counter += 1

    if perm_test:
        data.append(perm_p_values)
        labels.append('Permutation Test')
        positions.append(position_counter)
        prop_perm = len([i for i in perm_p_values if i < signif_alpha]) / len(perm_p_values)
        proportions.append(prop_perm)
        position_counter += 1

    if t_test:
        data.append(tstt_p_values)
        labels.append('Two-Sample T-Test')
        positions.append(position_counter)
        prop_ttest = len([i for i in tstt_p_values if i < signif_alpha]) / len(tstt_p_values)
        proportions.append(prop_ttest)
        position_counter += 1

    if data:
        # Violin plots
        vp = ax2.violinplot(data, positions=positions, showmeans=False, showmedians=False, showextrema=False)

        # Customize violin plots
        for pc in vp['bodies']:
            pc.set_alpha(0.5)
            pc.set_edgecolor('black')
            pc.set_facecolor('lightblue')

        # Box plots
        bp = ax2.boxplot(data, positions=positions, widths=0.1, patch_artist=True,
                         boxprops=dict(facecolor='white', color='black'),
                         medianprops=dict(color='black'),
                         whiskerprops=dict(color='black'),
                         capprops=dict(color='black'))

        # Overlay scatter plots
        colors = ['red'] * len(data)
        for i, y in enumerate(data):
            x = np.random.normal(positions[i], 0.05, size=len(y))
            ax2.plot(x, y, '.', color=colors[i], alpha=0.5)

        # Add significance line
        ax2.axhline(y=signif_alpha, color='red', linestyle='--', label=f'Alpha = {signif_alpha}')

        # Include proportions in x-axis labels
        labels_with_prop = [f"{label}\nProp: {prop:.2%}" for label, prop in zip(labels, proportions)]

        # Set labels and title
        ax2.set_xticks(positions)
        ax2.set_xticklabels(labels_with_prop, rotation=45)
        ax2.set_title("P-values with Violin and Box Plots")
        ax2.set_ylabel("P-value")
        ax2.legend()
    else:
        ax2.text(0.5, 0.5, 'No tests selected.', transform=ax2.transAxes,
                 fontsize=14, ha='center', va='center')
        ax2.axis('off')

    plt.tight_layout()
    plt.show()

import ipywidgets as widgets
from ipywidgets import interact

interact(plot_interactive_tests, 
         mean1=widgets.FloatSlider(value=50, min=0, max=100, step=1, description='Mean 1'),
         delta=widgets.FloatSlider(value=0, min=-50, max=50, step=1, description='Delta'),
         std_dev1=widgets.FloatSlider(value=10, min=1, max=30, step=1, description='Std Dev 1'),
         std_dev2=widgets.FloatSlider(value=10, min=1, max=30, step=1, description='Std Dev 2'),
         size=widgets.IntSlider(value=50, min=10, max=200, step=5, description='Sample Size'),
         runs=widgets.IntSlider(value=10, min=10, max=200, step=10, description='Runs'),
         signif_alpha=widgets.FloatSlider(value=0.05, min=0, max=1, step=0.01, description='Alpha'),
         mw_test=widgets.ToggleButton(value=True, description='Mann-Whitney'),
         waerden_test=widgets.ToggleButton(value=True, description='Waerden'),
         median_t=widgets.ToggleButton(value=True, description='Median Test'),
         fy_test=widgets.ToggleButton(value=True, description='FYTH'),
         perm_test=widgets.ToggleButton(value=True, description='Permutation Test'),
         t_test=widgets.ToggleButton(value=True, description='Two-Sample T-Test'))


interactive(children=(FloatSlider(value=50.0, description='Mean 1', step=1.0), FloatSlider(value=0.0, descript…

<function __main__.plot_interactive_tests(mean1=50, delta=0, std_dev1=10, std_dev2=10, size=100, signif_alpha=0.05, runs=100, mw_test=True, waerden_test=True, median_t=True, fy_test=True, perm_test=True, t_test=True)>

### book page 970

# Poisson distribution

In [43]:
def plot_interactive_tests(mean1=0, mean2=0, size=100):
    np.random.seed(42)

    mean1 = 10 ** mean1
    mean2 = 10 ** mean2
    
    sample1 = np.random.poisson(lam=mean1, size=size)
    sample2 = np.random.poisson(lam=mean2, size=size)

    stat_mw, p_value_mw = mannwhitneyu(sample1, sample2, alternative='two-sided')
    
    combined = np.concatenate([sample1, sample2])
    ranks = rankdata(combined)
    normal_scores = norm.ppf(ranks / (len(combined) + 1))

    scores1, scores2 = normal_scores[:size], normal_scores[size:]
    mean_diff = np.abs(np.mean(scores1) - np.mean(scores2))
    pooled_std = np.sqrt(np.var(scores1, ddof=1) / size + np.var(scores2, ddof=1) / size)
    stat_waerden = mean_diff / pooled_std
    p_value_waerden = 2 * (1 - norm.cdf(stat_waerden))


    plt.figure(figsize=(10, 6))
    plt.hist(sample1, bins='auto', alpha=0.7, label=f'Sample 1 (Mean={mean1:.3f})', color='blue', edgecolor='black')
    plt.hist(sample2, bins='auto', alpha=0.7, label=f'Sample 2 (Mean={mean2:.3f})', color='orange', edgecolor='black')
    plt.axvline(np.mean(sample1), color='blue', linestyle='dashed', linewidth=1)
    plt.axvline(np.mean(sample2), color='orange', linestyle='dashed', linewidth=1)
    plt.title("Histogram of Sample 1 and Sample 2")
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.legend()
    
    plt.figtext(0.15, 0.75, f"Wilcoxon Mann-Whitney Test\nStatistic: {stat_mw:.2f}\nP-value: {p_value_mw:.4f}", 
                fontsize=10, color="black", bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))
    plt.figtext(0.15, 0.65, f"Van der Waerden Test\nStatistic: {stat_waerden:.2f}\nP-value: {p_value_waerden:.4f}", 
                fontsize=10, color="black", bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))
    plt.show()

interact(plot_interactive_tests, 
         mean1=widgets.FloatSlider(value=0, min=-10, max=10, step=0.1, description='Mean 1'),
         mean2=widgets.FloatSlider(value=0, min=-10, max=10, step=0.1, description='Mean 2'),
         size=widgets.IntSlider(value=300, min=100, max=500, step=50, description='Sample Size')
         )


interactive(children=(FloatSlider(value=0.0, description='Mean 1', max=10.0, min=-10.0), FloatSlider(value=0.0…

<function __main__.plot_interactive_tests(mean1=0, mean2=0, size=100)>

# Mixture distributions

In [39]:
def generate_mixture_samples(mean1, std_dev1, mean2, std_dev2, size, threshold=0.5):
    # Generate samples from both distributions
    sample1 = np.random.normal(loc=mean1, scale=std_dev1, size=size)
    sample2 = np.random.normal(loc=mean2, scale=std_dev2, size=size)

    # Generate random probabilities to determine which distribution to choose
    random_probs = np.random.uniform(0, 1, size)

    # Choose samples based on the threshold
    samples = np.where(random_probs > threshold, sample1, sample2)

    return samples

# Example usage:
mean1, std_dev1 = 50, 10
mean2, std_dev2 = 70, 15
size = 10
prob = 0.7
threshold = 0.5

mixture_samples = generate_mixture_samples(mean1, std_dev1, mean2, std_dev2, size, threshold)
print(mixture_samples)


[69.12391219 39.85690732 68.76124807 84.4227826  54.06424988 67.12086703
 53.47270586 53.43230581 52.23069044 91.05931255]


In [40]:
def plot_interactive_tests_mixture(
    mean1=50, delta=0, std_dev1=10, std_dev2=10, size=100, signif_alpha=0.05, 
    runs=100, mixture_thresh=0.1):
    np.random.seed(42)

    mean2 = mean1 + delta
    mw_p_values, waerden_p_values, median_test_p_values = [], [], []
    fy_p_values, perm_p_values, tstt_p_values = [], [], []
    
    for _ in range(runs):
        sample1 = generate_mixture_samples(mean1, std_dev1, mean1 + 3*std_dev1, std_dev1*0.1, size, mixture_thresh)
        sample2 = generate_mixture_samples(mean2, std_dev2, mean2 + 3*std_dev2, std_dev2*0.1, size, mixture_thresh)
    
        stat_mw, p_value_mw = mannwhitneyu(sample1, sample2, alternative='two-sided')
        combined = np.concatenate([sample1, sample2])
        ranks = rankdata(combined)
        normal_scores = norm.ppf(ranks / (len(combined) + 1))
    
        scores1, scores2 = normal_scores[:size], normal_scores[size:]
        mean_diff = np.abs(np.mean(scores1) - np.mean(scores2))
        pooled_std = np.sqrt(np.var(scores1, ddof=1) / size + np.var(scores2, ddof=1) / size)
        stat_waerden = mean_diff / pooled_std
        p_value_waerden = 2 * (1 - norm.cdf(stat_waerden))
        
        # _, median_p_value = median_test(sample1, sample2)

        # _, fy_p_value = fisher_yates_terry_hoeffding_test(sample1, sample2)

        # perm_p_value, _ = permutation_test(sample1, sample2, alternative='two-sided')

        # _, tstt_p_value = two_sample_t_test(sample1, sample2, equal_var=std_dev1==std_dev2, alternative='two-sided')

        mw_p_values.append(p_value_mw)
        waerden_p_values.append(p_value_waerden)
        # median_test_p_values.append(median_p_value)
        # fy_p_values.append(fy_p_value)
        # perm_p_values.append(perm_p_value)
        # tstt_p_values.append(tstt_p_value)

    fig = plt.figure(figsize=(15, 6))

    # First subplot: Histograms of samples
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.hist(sample1, bins='auto', alpha=0.7, label=f'Sample 1 (Mean={mean1})', color='blue', edgecolor='black')
    ax1.hist(sample2, bins='auto', alpha=0.7, label=f'Sample 2 (Mean={mean2})', color='orange', edgecolor='black')
    ax1.axvline(np.mean(sample1), color='blue', linestyle='dashed', linewidth=1)
    ax1.axvline(np.mean(sample2), color='orange', linestyle='dashed', linewidth=1)
    
    ax1.set_title("Histogram of Sample 1 and Sample 2")
    ax1.set_xlabel("Value")
    ax1.set_ylabel("Frequency")
    ax1.legend()

    # Add text boxes
    ax1.text(0.05, 0.95, f"Mann-Whitney Test\nStatistic: {stat_mw:.2f}\nP-value: {p_value_mw:.4f}\nSignificant: {p_value_mw < signif_alpha}",
             transform=ax1.transAxes, fontsize=10, verticalalignment='top', bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))
    ax1.text(0.05, 0.75, f"Van der Waerden Test\nStatistic: {stat_waerden:.2f}\nP-value: {p_value_waerden:.4f}\nSignificant: {p_value_waerden < signif_alpha}",
             transform=ax1.transAxes, fontsize=10, verticalalignment='top', bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))

    # Second subplot: Violin plots, box plots, and scatter overlay
    ax2 = fig.add_subplot(1, 2, 2)
    data = [mw_p_values, waerden_p_values]
    labels = ['Mann-Whitney', 'Waerden']
    positions = [1, 2]
    
    # Violin plots
    vp = ax2.violinplot(data, positions=positions, showmeans=False, showmedians=False, showextrema=False)
    
    # Customize violin plots
    for i, pc in enumerate(vp['bodies']):
        pc.set_alpha(0.5)
        pc.set_edgecolor('black')
        pc.set_facecolor('lightblue')
    
    # Box plots
    bp = ax2.boxplot(data, positions=positions, widths=0.1, patch_artist=True, 
                     boxprops=dict(facecolor='white', color='black'),
                     medianprops=dict(color='black'),
                     whiskerprops=dict(color='black'),
                     capprops=dict(color='black'))
    
    # Overlay scatter plots
    colors = ['red', 'red']
    for i in range(len(data)):
        y = data[i]
        x = np.random.normal(positions[i], 0.05, size=len(y))
        ax2.plot(x, y, '.', color=colors[i], alpha=0.5)
    
    # Add significance line
    ax2.axhline(y=signif_alpha, color='red', linestyle='--', label=f'Alpha = {signif_alpha}')

    # Set labels and title
    ax2.set_xticks(positions)
    ax2.set_xticklabels(labels)
    ax2.set_title("P-values with Violin and Box Plots")
    ax2.set_ylabel("P-value")
    ax2.legend()

    plt.tight_layout()
    plt.show()

interact(plot_interactive_tests_mixture, 
         mean1=widgets.FloatSlider(value=50, min=0, max=100, step=1, description='Mean 1'),
         delta=widgets.FloatSlider(value=0, min=-100, max=100, step=1, description='Delta'),
         std_dev1=widgets.FloatSlider(value=10, min=1, max=30, step=1, description='Std Dev 1'),
         std_dev2=widgets.FloatSlider(value=10, min=1, max=30, step=1, description='Std Dev 2'),
         size=widgets.IntSlider(value=300, min=100, max=500, step=50, description='Sample Size'),
         runs=widgets.IntSlider(value=100, min=100, max=500, step=50, description='Runs'),
         signif_alpha=widgets.FloatSlider(value=0.05, min=0, max=1, step=0.01, description='Alpha'),
         mixture_thresh=widgets.FloatSlider(value=0.1, min=0, max=1, step=0.1, description='Mixture percent'))


interactive(children=(FloatSlider(value=50.0, description='Mean 1', step=1.0), FloatSlider(value=0.0, descript…

<function __main__.plot_interactive_tests_mixture(mean1=50, delta=0, std_dev1=10, std_dev2=10, size=100, signif_alpha=0.05, runs=100, mixture_thresh=0.1)>

# T distribution

In [14]:
def plot_interactive_tests(mean1=50, mean2=55, size=100, df=2):
    np.random.seed(42)

    sample1 = np.random.standard_t(df=df, size=size) * mean1 / 10 + mean1
    sample2 = np.random.standard_t(df=df, size=size) * mean2 / 10 + mean2

    stat_mw, p_value_mw = mannwhitneyu(sample1, sample2, alternative='two-sided')
    
    combined = np.concatenate([sample1, sample2])
    ranks = rankdata(combined)
    normal_scores = norm.ppf(ranks / (len(combined) + 1))

    scores1, scores2 = normal_scores[:size], normal_scores[size:]
    mean_diff = np.abs(np.mean(scores1) - np.mean(scores2))
    pooled_std = np.sqrt(np.var(scores1, ddof=1) / size + np.var(scores2, ddof=1) / size)
    stat_waerden = mean_diff / pooled_std
    p_value_waerden = 2 * (1 - norm.cdf(stat_waerden))


    plt.figure(figsize=(10, 6))
    plt.hist(sample1, bins='auto', alpha=0.7, label=f'Sample 1 (Mean={mean1})', color='blue', edgecolor='black')
    plt.hist(sample2, bins='auto', alpha=0.7, label=f'Sample 2 (Mean={mean2})', color='orange', edgecolor='black')
    plt.axvline(np.mean(sample1), color='blue', linestyle='dashed', linewidth=1)
    plt.axvline(np.mean(sample2), color='orange', linestyle='dashed', linewidth=1)
    plt.title("Histogram of Sample 1 and Sample 2")
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.legend()
    
    plt.figtext(0.15, 0.75, f"Wilcoxon Mann-Whitney Test\nStatistic: {stat_mw:.2f}\nP-value: {p_value_mw:.4f}", 
                fontsize=10, color="black", bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))
    plt.figtext(0.15, 0.65, f"Van der Waerden Test\nStatistic: {stat_waerden:.2f}\nP-value: {p_value_waerden:.4f}", 
                fontsize=10, color="black", bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))
    

    plt.show()

interact(plot_interactive_tests, 
        mean1=widgets.FloatSlider(value=5, min=1, max=60, step=0.1, description='Mean 1'),
        mean2=widgets.FloatSlider(value=10, min=1, max=60, step=0.1, description='Mean 2'),
        size=widgets.IntSlider(value=300, min=100, max=500, step=50, description='Sample Size'),
        df=widgets.IntSlider(value=20, min=1, max=50, step=1, description='Degree of freedom'),
        )


interactive(children=(FloatSlider(value=5.0, description='Mean 1', max=60.0, min=1.0), FloatSlider(value=10.0,…

<function __main__.plot_interactive_tests(mean1=50, mean2=55, size=100, df=2)>

# Cauchy

In [15]:
def plot_interactive_tests(loc1=0, scale1=1, loc2=0, scale2=1, size=100):
    np.random.seed(42)

    sample1 = cauchy.rvs(loc=loc1, scale=scale1, size=size)
    sample2 = cauchy.rvs(loc=loc2, scale=scale2, size=size)

    stat_mw, p_value_mw = mannwhitneyu(sample1, sample2, alternative='two-sided')
    
    combined = np.concatenate([sample1, sample2])
    ranks = rankdata(combined)
    normal_scores = norm.ppf(ranks / (len(combined) + 1))

    scores1, scores2 = normal_scores[:size], normal_scores[size:]
    mean_diff = np.abs(np.mean(scores1) - np.mean(scores2))
    pooled_std = np.sqrt(np.var(scores1, ddof=1) / size + np.var(scores2, ddof=1) / size)
    stat_waerden = mean_diff / pooled_std
    p_value_waerden = 2 * (1 - norm.cdf(stat_waerden))

    plt.figure(figsize=(10, 6))
    plt.hist(sample1, bins='auto', alpha=0.7, label=f'Sample 1 (Loc={loc1}, Scale={scale1})', color='blue', edgecolor='black')
    plt.hist(sample2, bins='auto', alpha=0.7, label=f'Sample 2 (Loc={loc2}, Scale={scale2})', color='orange', edgecolor='black')
    plt.axvline(np.median(sample1), color='blue', linestyle='dashed', linewidth=1)
    plt.axvline(np.median(sample2), color='orange', linestyle='dashed', linewidth=1)
    plt.title("Histogram of Sample 1 and Sample 2")
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.legend()
    
    plt.figtext(0.15, 0.75, f"Wilcoxon Mann-Whitney Test\nStatistic: {stat_mw:.2f}\nP-value: {p_value_mw:.4f}", 
                fontsize=10, color="black", bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))
    plt.figtext(0.15, 0.65, f"Van der Waerden Test\nStatistic: {stat_waerden:.2f}\nP-value: {p_value_waerden:.4f}", 
                fontsize=10, color="black", bbox=dict(facecolor="white", edgecolor="black", boxstyle="round,pad=0.5"))

    plt.show()

interact(plot_interactive_tests, 
         loc1=widgets.FloatSlider(value=0, min=-10, max=10, step=0.1, description='Loc 1'),
         scale1=widgets.FloatSlider(value=1, min=0.1, max=10, step=0.1, description='Scale 1'),
         loc2=widgets.FloatSlider(value=0, min=-10, max=10, step=0.1, description='Loc 2'),
         scale2=widgets.FloatSlider(value=1, min=0.1, max=10, step=0.1, description='Scale 2'),
         size=widgets.IntSlider(value=100, min=50, max=500, step=50, description='Sample Size'))

interactive(children=(FloatSlider(value=0.0, description='Loc 1', max=10.0, min=-10.0), FloatSlider(value=1.0,…

<function __main__.plot_interactive_tests(loc1=0, scale1=1, loc2=0, scale2=1, size=100)>

### Wilcoxon impl

In [32]:
import numpy as np
from scipy.stats import mannwhitneyu
from math import erf, sqrt

# Manual Wilcoxon Rank Sum Test Implementation
def wilcoxon_rank_sum_test_manual(X, Y):
    # Combine the samples
    combined = np.concatenate([X, Y])
    labels = np.array([1] * len(X) + [2] * len(Y))  # Label sample X with 1, Y with 2
    
    # Sort combined data and assign ranks
    sorted_indices = np.argsort(combined)
    ranks = np.zeros_like(combined, dtype=float)
    
    # Assign ranks (average ranks for ties)
    i = 0
    while i < len(combined):
        start = i
        while i + 1 < len(combined) and combined[sorted_indices[i]] == combined[sorted_indices[i + 1]]:
            i += 1
        rank = (start + i + 1) / 2.0  # Average rank for tied values
        for j in range(start, i + 1):
            ranks[sorted_indices[j]] = rank
        i += 1

    # Calculate W (sum of ranks for sample X)
    W_X = np.sum(ranks[labels == 1])
    
    # Expected value and variance of W under H0
    n, m = len(X), len(Y)
    mu_W = n * (n + m + 1) / 2
    sigma_W = np.sqrt(n * m * (n + m + 1) / 12)
    
    # Calculate z-score for the test statistic
    z = (W_X - mu_W) / sigma_W
    
    # Convert z-score to p-value for a two-tailed test
    p_value_manual = 2 * (1 - 0.5 * (1 + erf(abs(z) / sqrt(2))))
    
    return W_X, mu_W, sigma_W, z, p_value_manual

# Wilcoxon Rank Sum Test using scipy
def wilcoxon_rank_sum_test_scipy(X, Y):
    u_statistic, p_value = mannwhitneyu(X, Y, alternative='two-sided')
    # Convert U to W for comparison
    W_converted = u_statistic + len(X) * (len(X) + 1) / 2
    return u_statistic, W_converted, p_value

# Comparison function
def compare_tests(X, Y):
    # Manual implementation
    W_X_manual, mu_W_manual, sigma_W_manual, z_manual, p_value_manual = wilcoxon_rank_sum_test_manual(X, Y)
    # Scipy implementation
    u_statistic_scipy, W_scipy_converted, p_value_scipy = wilcoxon_rank_sum_test_scipy(X, Y)
    
    # Display results
    print("Manual Wilcoxon Test Results")
    print(f"W: {W_X_manual}")
    print(f"Expected W: {mu_W_manual}")
    print(f"Standard Deviation of W: {sigma_W_manual}")
    print(f"z-score: {z_manual}")
    print(f"p-value (from z-score): {p_value_manual}")
    
    print("\nScipy Wilcoxon Test Results")
    print(f"U-statistic: {u_statistic_scipy}")
    print(f"Converted W from U: {W_scipy_converted}")
    print(f"p-value (scipy): {p_value_scipy}")

# Example usage
X = np.array([2.5, 3.6, 3.1, 4.4, 3.2])  # Sample X
Y = np.array([1.2, 2.1, 2.4, 2.3, 3.5])  # Sample Y

# Run comparison
compare_tests(X, Y)


Manual Wilcoxon Test Results
W: 34.5
Expected W: 27.5
Standard Deviation of W: 4.7871355387816905
z-score: 1.462252310027862
p-value (from z-score): 0.14367208180696034

Scipy Wilcoxon Test Results
U-statistic: 22.0
Converted W from U: 37.0
p-value (scipy): 0.05555555555555555


In [13]:
import numpy as np
from collections import namedtuple
from scipy import special
from scipy import stats

def _order_ranks(ranks, j):
    # Reorder ascending order `ranks` according to `j`
    ordered_ranks = np.empty(j.shape, dtype=ranks.dtype)
    np.put_along_axis(ordered_ranks, j, ranks, axis=-1)
    return ordered_ranks

def _rankdata(x, method, return_ties=False):
    shape = x.shape

    kind = 'mergesort' if method == 'ordinal' else 'quicksort'
    j = np.argsort(x, axis=-1, kind=kind)
    ordinal_ranks = np.broadcast_to(np.arange(1, shape[-1]+1, dtype=int), shape)

    if method == 'ordinal':
        return _order_ranks(ordinal_ranks, j)  # never return ties

    y = np.take_along_axis(x, j, axis=-1)
    i = np.concatenate([np.ones(shape[:-1] + (1,), dtype=np.bool_),
                       y[..., :-1] != y[..., 1:]], axis=-1)

    indices = np.arange(y.size)[i.ravel()]
    counts = np.diff(indices, append=y.size)

    if method == 'min':
        ranks = ordinal_ranks[i]
    elif method == 'max':
        ranks = ordinal_ranks[i] + counts - 1
    elif method == 'average':
        ranks = ordinal_ranks[i] + (counts - 1)/2
    elif method == 'dense':
        ranks = np.cumsum(i, axis=-1)[i]

    ranks = np.repeat(ranks, counts).reshape(shape)
    ranks = _order_ranks(ranks, j)

    if return_ties:
        t = np.zeros(shape, dtype=float)
        t[i] = counts
        return ranks, t
    return ranks

def _broadcast_concatenate(x, y, axis):
    '''Broadcast then concatenate arrays, leaving concatenation axis last'''
    x = np.moveaxis(x, axis, -1)
    y = np.moveaxis(y, axis, -1)
    z = np.broadcast(x[..., 0], y[..., 0])
    x = np.broadcast_to(x, z.shape + (x.shape[-1],))
    y = np.broadcast_to(y, z.shape + (y.shape[-1],))
    z = np.concatenate((x, y), axis=-1)
    return x, y, z


class _MWU:
    '''Distribution of MWU statistic under the null hypothesis'''

    def __init__(self, n1, n2):
        self._reset(n1, n2)

    def set_shapes(self, n1, n2):
        n1, n2 = min(n1, n2), max(n1, n2)
        if (n1, n2) == (self.n1, self.n2):
            return

        self.n1 = n1
        self.n2 = n2
        self.s_array = np.zeros(0, dtype=int)
        self.configurations = np.zeros(0, dtype=np.uint64)

    def reset(self):
        self._reset(self.n1, self.n2)

    def _reset(self, n1, n2):
        self.n1 = None
        self.n2 = None
        self.set_shapes(n1, n2)

    def pmf(self, k):
        pmfs = self.build_u_freqs_array(np.max(k))
        return pmfs[k]

    def cdf(self, k):
        '''Cumulative distribution function'''
        pmfs = self.build_u_freqs_array(np.max(k))
        cdfs = np.cumsum(pmfs)
        return cdfs[k]

    def sf(self, k):
        '''Survival function'''
        kc = np.asarray(self.n1*self.n2 - k)  # complement of k
        i = k < kc
        if np.any(i):
            kc[i] = k[i]
            cdfs = np.asarray(self.cdf(kc))
            cdfs[i] = 1. - cdfs[i] + self.pmf(kc[i])
        else:
            cdfs = np.asarray(self.cdf(kc))
        return cdfs[()]

    # build_sigma_array and build_u_freqs_array adapted from code
    # by @toobaz with permission. Thanks to @andreasloe for the suggestion.
    # See https://github.com/scipy/scipy/pull/4933#issuecomment-1898082691
    def build_sigma_array(self, a):
        n1, n2 = self.n1, self.n2
        if a + 1 <= self.s_array.size:
            return self.s_array[1:a+1]

        s_array = np.zeros(a + 1, dtype=int)

        for d in np.arange(1, n1 + 1):
            # All multiples of d, except 0:
            indices = np.arange(d, a + 1, d)
            # \epsilon_d = 1:
            s_array[indices] += d

        for d in np.arange(n2 + 1, n2 + n1 + 1):
            # All multiples of d, except 0:
            indices = np.arange(d, a + 1, d)
            # \epsilon_d = -1:
            s_array[indices] -= d

        # We don't need 0:
        self.s_array = s_array
        return s_array[1:]

    def build_u_freqs_array(self, maxu):
        """
        Build all the array of frequencies for u from 0 to maxu.
        Assumptions:
          n1 <= n2
          maxu <= n1 * n2 / 2
        """
        n1, n2 = self.n1, self.n2
        total = special.binom(n1 + n2, n1)

        if maxu + 1 <= self.configurations.size:
            return self.configurations[:maxu + 1] / total

        s_array = self.build_sigma_array(maxu)

        # Start working with ints, for maximum precision and efficiency:
        configurations = np.zeros(maxu + 1, dtype=np.uint64)
        configurations_is_uint = True
        uint_max = np.iinfo(np.uint64).max
        # How many ways to have U=0? 1
        configurations[0] = 1

        for u in np.arange(1, maxu + 1):
            coeffs = s_array[u - 1::-1]
            new_val = np.dot(configurations[:u], coeffs) / u
            if new_val > uint_max and configurations_is_uint:
                # OK, we got into numbers too big for uint64.
                # So now we start working with floats.
                # By doing this since the beginning, we would have lost precision.
                # (And working on python long ints would be unbearably slow)
                configurations = configurations.astype(float)
                configurations_is_uint = False
            configurations[u] = new_val

        self.configurations = configurations
        return configurations / total


_mwu_state = _MWU(0, 0)


def _get_mwu_z(U, n1, n2, t, axis=0, continuity=True):
    '''Standardized MWU statistic'''
    # Follows mannwhitneyu [2]
    mu = n1 * n2 / 2
    n = n1 + n2

    # Tie correction according to [2], "Normal approximation and tie correction"
    # "A more computationally-efficient form..."
    tie_term = (t**3 - t).sum(axis=-1)
    s = np.sqrt(n1*n2/12 * ((n + 1) - tie_term/(n*(n-1))))

    numerator = U - mu

    # Continuity correction.
    # Because SF is always used to calculate the p-value, we can always
    # _subtract_ 0.5 for the continuity correction. This always increases the
    # p-value to account for the rest of the probability mass _at_ q = U.
    if continuity:
        numerator -= 0.5

    # no problem evaluating the norm SF at an infinity
    with np.errstate(divide='ignore', invalid='ignore'):
        z = numerator / s
    return z


def _mwu_input_validation(x, y, use_continuity, alternative, axis, method):
    ''' Input validation and standardization for mannwhitneyu '''
    # Would use np.asarray_chkfinite, but infs are OK
    x, y = np.atleast_1d(x), np.atleast_1d(y)
    if np.isnan(x).any() or np.isnan(y).any():
        raise ValueError('`x` and `y` must not contain NaNs.')
    if np.size(x) == 0 or np.size(y) == 0:
        raise ValueError('`x` and `y` must be of nonzero size.')

    bools = {True, False}
    if use_continuity not in bools:
        raise ValueError(f'`use_continuity` must be one of {bools}.')

    alternatives = {"two-sided", "less", "greater"}
    alternative = alternative.lower()
    if alternative not in alternatives:
        raise ValueError(f'`alternative` must be one of {alternatives}.')

    axis_int = int(axis)
    if axis != axis_int:
        raise ValueError('`axis` must be an integer.')

    if not isinstance(method, stats.PermutationMethod):
        methods = {"asymptotic", "exact", "auto"}
        method = method.lower()
        if method not in methods:
            raise ValueError(f'`method` must be one of {methods}.')

    return x, y, use_continuity, alternative, axis_int, method


def _mwu_choose_method(n1, n2, ties):
    """Choose method 'asymptotic' or 'exact' depending on input size, ties"""

    # if both inputs are large, asymptotic is OK
    if n1 > 8 and n2 > 8:
        return "asymptotic"

    # if there are any ties, asymptotic is preferred
    if ties:
        return "asymptotic"

    return "exact"


MannwhitneyuResult = namedtuple('MannwhitneyuResult', ('statistic', 'pvalue'))


def mannwhitneyu(x, y, use_continuity=True, alternative="two-sided",
                 axis=0, method="auto"):

    x, y, use_continuity, alternative, axis_int, method = (
        _mwu_input_validation(x, y, use_continuity, alternative, axis, method))

    x, y, xy = _broadcast_concatenate(x, y, axis)

    n1, n2 = x.shape[-1], y.shape[-1]

    # Follows [2]
    ranks, t = _rankdata(xy, 'average', return_ties=True)  # method 2, step 1
    R1 = ranks[..., :n1].sum(axis=-1)                      # method 2, step 2
    U1 = R1 - n1*(n1+1)/2                                  # method 2, step 3
    U2 = n1 * n2 - U1                                      # as U1 + U2 = n1 * n2

    if alternative == "greater":
        U, f = U1, 1  # U is the statistic to use for p-value, f is a factor
    elif alternative == "less":
        U, f = U2, 1  # Due to symmetry, use SF of U2 rather than CDF of U1
    else:
        U, f = np.maximum(U1, U2), 2  # multiply SF by two for two-sided test

    if method == "auto":
        method = _mwu_choose_method(n1, n2, np.any(t > 1))

    if method == "exact":
        _mwu_state.set_shapes(n1, n2)
        p = _mwu_state.sf(U.astype(int))
    elif method == "asymptotic":
        z = _get_mwu_z(U, n1, n2, t, continuity=use_continuity)
        p = stats.norm.sf(z)
    else:  # `PermutationMethod` instance (already validated)
        def statistic(x, y, axis):
            return mannwhitneyu(x, y, use_continuity=use_continuity,
                                alternative=alternative, axis=axis,
                                method="asymptotic").statistic

        res = stats.permutation_test((x, y), statistic, axis=axis,
                                     **method._asdict(), alternative=alternative)
        p = res.pvalue
        f = 1

    p *= f

    # Ensure that test statistic is not greater than 1
    # This could happen for exact test when U = m*n/2
    p = np.clip(p, 0, 1)

    return MannwhitneyuResult(U1, p)

In [14]:
np.random.seed(0)
sample1 = np.random.normal(loc=50, scale=10, size=300)
sample2 = np.random.normal(loc=60, scale=20, size=300)

mannwhitneyu(sample1, sample2, alternative='two-sided')

MannwhitneyuResult(statistic=32960.0, pvalue=1.4217284932393384e-08)

In [15]:
np.random.seed(0)
sample1 = np.random.normal(loc=50, scale=10, size=300)
sample2 = np.random.normal(loc=60, scale=20, size=300)

mannwhitneyu(sample1, sample2, alternative='two-sided')

MannwhitneyuResult(statistic=32960.0, pvalue=1.4217284932393384e-08)

In [7]:
import numpy as np
from collections import namedtuple
from scipy import special, stats
# taken from scipy's library

def _order_ranks(ranks, indices):
    """Reorder ranks according to indices."""
    ordered_ranks = np.empty_like(ranks)
    np.put_along_axis(ordered_ranks, indices, ranks, axis=-1)
    return ordered_ranks


def _rankdata(x, method, return_ties=False):
    """Rank data with different ranking methods."""
    shape = x.shape
    kind = 'mergesort' if method == 'ordinal' else 'quicksort'
    indices = np.argsort(x, axis=-1, kind=kind)
    ranks = np.broadcast_to(np.arange(1, shape[-1] + 1), shape)

    if method == 'ordinal':
        return _order_ranks(ranks, indices)

    y = np.take_along_axis(x, indices, axis=-1)
    diff = np.concatenate([np.ones_like(y[..., :1], dtype=bool), y[..., :-1] != y[..., 1:]], axis=-1)
    counts = np.diff(np.flatnonzero(diff), append=y.size)

    if method == 'min':
        ranks = ranks[diff]
    elif method == 'max':
        ranks = ranks[diff] + counts - 1
    elif method == 'average':
        ranks = ranks[diff] + (counts - 1) / 2
    elif method == 'dense':
        ranks = np.cumsum(diff, axis=-1)[diff]

    ranks = np.repeat(ranks, counts).reshape(shape)
    ranks = _order_ranks(ranks, indices)

    if return_ties:
        ties = np.zeros_like(x, dtype=float)
        ties[diff] = counts
        return ranks, ties

    return ranks


def _broadcast_concatenate(x, y, axis):
    """Broadcast and concatenate arrays along the last axis."""
    x, y = np.moveaxis(x, axis, -1), np.moveaxis(y, axis, -1)
    shape = np.broadcast(x[..., 0], y[..., 0]).shape
    x = np.broadcast_to(x, shape + (x.shape[-1],))
    y = np.broadcast_to(y, shape + (y.shape[-1],))
    return x, y, np.concatenate((x, y), axis=-1)


class _MWU:
    """Distribution of MWU statistic under the null hypothesis."""
    def __init__(self, n1, n2):
        self.n1, self.n2 = None, None
        self.set_shapes(n1, n2)

    def set_shapes(self, n1, n2):
        self.n1, self.n2 = sorted((n1, n2))
        self.s_array = np.zeros(0, dtype=int)
        self.configurations = np.zeros(0, dtype=np.uint64)

    def pmf(self, k):
        return self._build_u_freqs_array(np.max(k))[k]

    def cdf(self, k):
        pmfs = self._build_u_freqs_array(np.max(k))
        return np.cumsum(pmfs)[k]

    def _build_u_freqs_array(self, maxu):
        """Build frequency array for MWU statistic."""
        total = special.binom(self.n1 + self.n2, self.n1)
        s_array = self._build_sigma_array(maxu)

        configurations = np.zeros(maxu + 1, dtype=np.uint64)
        configurations[0] = 1

        for u in range(1, maxu + 1):
            coeffs = s_array[u - 1::-1]
            configurations[u] = np.dot(configurations[:u], coeffs) // u

        self.configurations = configurations
        return configurations / total

    def _build_sigma_array(self, maxa):
        """Build sigma array for MWU statistic."""
        if maxa < len(self.s_array):
            return self.s_array[:maxa + 1]

        s_array = np.zeros(maxa + 1, dtype=int)
        for d in range(1, self.n1 + 1):
            s_array[d::d] += d
        for d in range(self.n2 + 1, self.n2 + self.n1 + 1):
            s_array[d::d] -= d

        self.s_array = s_array
        return s_array


def _mwu_choose_method(n1, n2, ties):
    """Choose MWU method based on input size and ties."""
    if n1 > 8 and n2 > 8 or ties:
        return "asymptotic"
    return "exact"


def _get_mwu_z(U, n1, n2, t, continuity=True):
    """Calculate standardized MWU statistic."""
    n, mu = n1 + n2, n1 * n2 / 2
    s = np.sqrt(n1 * n2 / 12 * ((n + 1) - (t**3 - t).sum() / (n * (n - 1))))
    return (U - mu - (0.5 if continuity else 0)) / s


MannwhitneyuResult = namedtuple('MannwhitneyuResult', ('statistic', 'pvalue'))


def mannwhitneyu(x, y, use_continuity=True, alternative="two-sided", axis=0, method="auto"):
    x, y, xy = _broadcast_concatenate(np.asarray(x), np.asarray(y), axis)
    n1, n2 = x.shape[-1], y.shape[-1]
    ranks, ties = _rankdata(xy, 'average', return_ties=True)
    U1 = ranks[..., :n1].sum(axis=-1) - n1 * (n1 + 1) / 2
    U2 = n1 * n2 - U1
    U = U1 if alternative == "greater" else U2 if alternative == "less" else np.maximum(U1, U2)

    if method == "auto":
        method = _mwu_choose_method(n1, n2, np.any(ties > 1))

    if method == "exact":
        _mwu_state = _MWU(n1, n2)
        p = _mwu_state.pmf(U.astype(int))
    elif method == "asymptotic":
        z = _get_mwu_z(U, n1, n2, ties, continuity=use_continuity)
        p = stats.norm.sf(z) * (2 if alternative == "two-sided" else 1)

    return MannwhitneyuResult(U1, np.clip(p, 0, 1))


In [8]:
np.random.seed(0)
sample1 = np.random.normal(loc=50, scale=10, size=300)
sample2 = np.random.normal(loc=60, scale=20, size=300)

mannwhitneyu(sample1, sample2, alternative='two-sided')

MannwhitneyuResult(statistic=32960.0, pvalue=1.4217284932393384e-08)

In [9]:
import numpy as np
from scipy.stats import norm, chi2

def van_der_waerden_test(*samples):
    """
    Perform the Van der Waerden test for k independent samples.

    Parameters:
    *samples : array-like
        Variable number of arrays, each representing an independent sample.

    Returns:
    statistic : float
        The test statistic.
    p_value : float
        The p-value of the test.
    """
    # Combine all samples into a single array
    data = np.concatenate(samples)
    n_total = len(data)
    k = len(samples)
    sample_sizes = [len(sample) for sample in samples]

    # Rank the combined data
    ranks = np.argsort(np.argsort(data)) + 1

    # Compute normal scores
    normal_scores = norm.ppf((ranks - 0.5) / n_total)

    # Calculate the sum of normal scores for each sample
    sum_scores = []
    start = 0
    for size in sample_sizes:
        sum_scores.append(np.sum(normal_scores[start:start + size]))
        start += size

    # Compute the test statistic
    sum_scores = np.array(sum_scores)
    n_i = np.array(sample_sizes)
    A_bar = sum_scores / n_i
    A_bar_total = np.sum(sum_scores) / n_total
    S_square = np.sum((normal_scores - A_bar_total) ** 2) / (n_total - 1)
    T = np.sum(n_i * (A_bar - A_bar_total) ** 2) / S_square

    # Calculate the p-value
    p_value = 1 - chi2.cdf(T, k - 1)

    return T, p_value


In [10]:
np.random.seed(0)
sample1 = np.random.normal(loc=50, scale=10, size=300)
sample2 = np.random.normal(loc=60, scale=20, size=300)
statistic, p_value = van_der_waerden_test(sample1, sample2)
print(f"Test Statistic: {statistic}")
print(f"P-Value: {p_value}")


Test Statistic: 26.299182518063674
P-Value: 2.9241242149868896e-07


In [11]:
import numpy as np
from scipy.stats import chi2_contingency

def median_test(*samples):
    """
    Perform the Median Test for k independent samples.

    Parameters:
    *samples : array-like
        Variable number of arrays, each representing an independent sample.

    Returns:
    statistic : float
        The test statistic.
    p_value : float
        The p-value of the test.
    """
    # Combine all samples into a single array
    data = np.concatenate(samples)
    grand_median = np.median(data)

    # Create a contingency table
    contingency_table = []
    for sample in samples:
        above_median = np.sum(sample > grand_median)
        below_or_equal_median = np.sum(sample <= grand_median)
        contingency_table.append([above_median, below_or_equal_median])

    # Perform the chi-squared test
    contingency_table = np.array(contingency_table)
    chi2, p_value, _, _ = chi2_contingency(contingency_table, correction=False)

    return chi2, p_value



In [12]:
np.random.seed(0)
sample1 = np.random.normal(loc=50, scale=10, size=300)
sample2 = np.random.normal(loc=60, scale=20, size=300)

statistic, p_value = median_test(sample1, sample2)
print(f"Test Statistic: {statistic}")
print(f"P-Value: {p_value}")


Test Statistic: 22.426666666666666
P-Value: 2.183216533714857e-06


In [13]:
import numpy as np
from scipy.stats import norm, chi2

def fisher_yates_terry_hoeffding_test(*samples):
    """
    Perform the Fisher-Yates-Terry-Hoeffding (FYTH) test for k independent samples.

    Parameters:
    *samples : array-like
        Variable number of arrays, each representing an independent sample.

    Returns:
    statistic : float
        The test statistic.
    p_value : float
        The p-value of the test.
    """
    # Combine all samples into a single array
    data = np.concatenate(samples)
    n_total = len(data)
    k = len(samples)
    sample_sizes = [len(sample) for sample in samples]

    # Rank the combined data
    ranks = np.argsort(np.argsort(data)) + 1

    # Compute expected normal scores
    expected_normal_scores = norm.ppf((ranks - 0.5) / n_total)

    # Calculate the sum of expected normal scores for each sample
    sum_scores = []
    start = 0
    for size in sample_sizes:
        sum_scores.append(np.sum(expected_normal_scores[start:start + size]))
        start += size

    # Compute the test statistic
    sum_scores = np.array(sum_scores)
    n_i = np.array(sample_sizes)
    A_bar = sum_scores / n_i
    A_bar_total = np.sum(sum_scores) / n_total
    S_square = np.sum((expected_normal_scores - A_bar_total) ** 2) / (n_total - 1)
    T = np.sum(n_i * (A_bar - A_bar_total) ** 2) / S_square

    # Calculate the p-value
    p_value = 1 - chi2.cdf(T, k - 1)

    return T, p_value

np.random.seed(0)
sample1 = np.random.normal(loc=50, scale=10, size=300)
sample2 = np.random.normal(loc=60, scale=20, size=300)

statistic, p_value = fisher_yates_terry_hoeffding_test(sample1, sample2)
print(f"Test Statistic: {statistic}")
print(f"P-Value: {p_value}")


Test Statistic: 26.299182518063674
P-Value: 2.9241242149868896e-07


In [24]:
import numpy as np
from scipy.stats import ttest_ind

def permutation_test(sample1, sample2, num_permutations=10000, alternative='two-sided'):
    """
    Perform a permutation test to compare two independent samples.

    Parameters:
    sample1 : array-like
        First sample data.
    sample2 : array-like
        Second sample data.
    num_permutations : int, optional
        Number of permutations to perform (default is 10,000).
    alternative : {'two-sided', 'greater', 'less'}, optional
        Defines the alternative hypothesis (default is 'two-sided').

    Returns:
    p_value : float
        The p-value of the test.
    observed_diff : float
        The observed difference in means between the two samples.
    """
    # Combine the data
    combined = np.concatenate([sample1, sample2])
    n1 = len(sample1)
    observed_diff = np.mean(sample1) - np.mean(sample2)

    # Generate permutation samples
    perm_diffs = np.zeros(num_permutations)
    for i in range(num_permutations):
        np.random.shuffle(combined)
        perm_sample1 = combined[:n1]
        perm_sample2 = combined[n1:]
        perm_diffs[i] = np.mean(perm_sample1) - np.mean(perm_sample2)

    # Calculate p-value
    if alternative == 'two-sided':
        p_value = np.mean(np.abs(perm_diffs) >= np.abs(observed_diff))
    elif alternative == 'greater':
        p_value = np.mean(perm_diffs >= observed_diff)
    elif alternative == 'less':
        p_value = np.mean(perm_diffs <= observed_diff)
    else:
        raise ValueError("alternative must be 'two-sided', 'greater', or 'less'")

    return p_value, observed_diff

np.random.seed(0)
sample1 = np.random.normal(loc=50, scale=10, size=300)
sample2 = np.random.normal(loc=60, scale=20, size=300)

p_value, observed_diff = permutation_test(sample1, sample2, alternative='two-sided')
print(f"Observed Difference: {observed_diff}")
print(f"P-Value: {p_value}")


Observed Difference: -6.9760915657594325
P-Value: 0.0


In [15]:
import numpy as np
from scipy.stats import ttest_ind

def two_sample_t_test(sample1, sample2, equal_var=True, alternative='two-sided'):
    """
    Perform a two-sample t-test to compare the means of two independent samples.

    Parameters:
    sample1 : array-like
        First sample data.
    sample2 : array-like
        Second sample data.
    equal_var : bool, optional
        If True (default), perform a standard independent 2 sample t-test that assumes equal population variances.
        If False, perform Welch’s t-test, which does not assume equal population variances.
    alternative : {'two-sided', 'less', 'greater'}, optional
        Defines the alternative hypothesis (default is 'two-sided').

    Returns:
    statistic : float
        The calculated t-statistic.
    p_value : float
        The p-value of the test.
    """
    # Perform the t-test
    t_stat, p_value = ttest_ind(sample1, sample2, equal_var=equal_var, alternative=alternative)
    return t_stat, p_value

np.random.seed(0)
sample1 = np.random.normal(loc=50, scale=10, size=300)
sample2 = np.random.normal(loc=60, scale=20, size=300)

t_stat, p_value = two_sample_t_test(sample1, sample2, equal_var=False, alternative='two-sided')
print(f"T-Statistic: {t_stat}")
print(f"P-Value: {p_value}")


T-Statistic: -5.3959889747439895
P-Value: 1.115761051138561e-07
